# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
import findspark
findspark.init('/home/ronit/spark-3.0.0-preview2-bin-hadoop2.7', edit_rc=True)
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('ship_crew').getOrCreate()

In [7]:
row_data = spark.read.csv('cruise_ship_info.csv', header=True,inferSchema=True)

In [8]:
row_data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [11]:
row_data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [10]:
row_data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [16]:
from pyspark.sql.functions import countDistinct,format_number

In [14]:
row_data.select(countDistinct('Ship_name'),countDistinct('Cruise_line')).show()

+-------------------------+---------------------------+
|count(DISTINCT Ship_name)|count(DISTINCT Cruise_line)|
+-------------------------+---------------------------+
|                      138|                         20|
+-------------------------+---------------------------+



In [18]:
row_data.select(
 format_number('Tonnage',2)).show()

+-------------------------+
|format_number(Tonnage, 2)|
+-------------------------+
|                    30.28|
|                    30.28|
|                    47.26|
|                   110.00|
|                   101.35|
|                    70.37|
|                    70.37|
|                    70.37|
|                    70.37|
|                   110.24|
|                   110.00|
|                    46.05|
|                    70.37|
|                    70.37|
|                    86.00|
|                   110.00|
|                    88.50|
|                    70.37|
|                    88.50|
|                    70.37|
+-------------------------+
only showing top 20 rows



In [21]:
row_data.select(['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']).describe().show()

+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|               158|               158|              158|              158|               158|              158|              158|
|   mean|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|                 4|             2.329|             0.66|             2.79|              0.33|             17.7|             0.59|
|    max|                48

In [22]:
from pyspark.ml.feature import StringIndexer

In [30]:
stringIndexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_indexed", stringOrderType="frequencyDesc")
model_string = stringIndexer.fit(row_data)
data = model_string.transform(row_data)

In [31]:
data.show(20)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_indexed|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|                1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|                1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|                1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.5

In [32]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_indexed']

In [28]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [33]:
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density','Cruise_line_indexed'],outputCol="features")

In [34]:
output = assembler.transform(data)

In [35]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_indexed: double (nullable = false)
 |-- features: vector (nullable = true)



In [36]:
final_data = output.select('features','crew')

In [37]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [38]:
training_data, test_data = final_data.randomSplit([0.7,0.3])

In [40]:
training_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               111|
|   mean| 7.985225225225236|
| stddev|3.6799685885138764|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [41]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                47|
|   mean| 7.342978723404257|
| stddev|3.0353904661750133|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [42]:
training_data.show()

+--------------------+-----+
|            features| crew|
+--------------------+-----+
|[4.0,220.0,54.0,1...| 21.0|
|[5.0,122.0,28.5,1...|  6.7|
|[5.0,133.5,39.59,...|13.13|
|[5.0,160.0,36.34,...| 13.6|
|[6.0,90.0,20.0,9....|  9.0|
|[6.0,93.0,23.94,9...|11.09|
|[6.0,110.23899999...| 11.5|
|[6.0,112.0,38.0,9...| 10.9|
|[7.0,89.6,25.5,9....| 9.87|
|[7.0,116.0,31.0,9...| 12.0|
|[7.0,158.0,43.7,1...| 13.6|
|[8.0,77.499,19.5,...|  9.0|
|[8.0,91.0,22.44,9...| 11.0|
|[8.0,110.0,29.74,...| 11.6|
|[9.0,59.058,17.0,...|  7.4|
|[9.0,81.0,21.44,9...| 10.0|
|[9.0,88.5,21.24,9...| 10.3|
|[9.0,90.09,25.01,...| 8.69|
|[9.0,105.0,27.2,8...|10.68|
|[9.0,110.0,29.74,...| 11.6|
+--------------------+-----+
only showing top 20 rows



In [43]:
lr = LinearRegression(labelCol = "crew")

In [44]:
lr_model = lr.fit(training_data)

In [45]:
test_results = lr_model.evaluate(test_data)

In [46]:
test_results.rootMeanSquaredError

0.9319014725262833

In [47]:
unlabeled_data = test_data.select('features')

In [48]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[5.0,86.0,21.04,9...|
|[5.0,115.0,35.74,...|
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[6.0,113.0,37.82,...|
|[6.0,158.0,43.7,1...|
|[9.0,85.0,19.68,9...|
|[10.0,81.76899999...|
|[10.0,90.09,25.01...|
|[10.0,138.0,31.14...|
|[11.0,58.6,15.66,...|
|[11.0,90.09,25.01...|
|[12.0,42.0,14.8,7...|
|[12.0,91.0,20.32,...|
|[12.0,138.0,31.14...|
|[14.0,33.0,4.9,5....|
|[14.0,63.0,14.4,7...|
|[14.0,77.104,20.0...|
|[14.0,83.0,17.5,9...|
|[14.0,138.0,31.14...|
+--------------------+
only showing top 20 rows



In [49]:
prediction = lr_model.transform(unlabeled_data)

In [50]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.395001930844552|
|[5.0,115.0,35.74,...|11.871380917818398|
|[6.0,30.276999999...| 4.283256040547585|
|[6.0,30.276999999...| 4.283256040547585|
|[6.0,113.0,37.82,...|11.741224099476058|
|[6.0,158.0,43.7,1...| 14.15074340528342|
|[9.0,85.0,19.68,9...| 9.392438544058782|
|[10.0,81.76899999...| 8.933747512678432|
|[10.0,90.09,25.01...| 9.083751152964798|
|[10.0,138.0,31.14...|13.122187597997524|
|[11.0,58.6,15.66,...| 7.455603079698362|
|[11.0,90.09,25.01...| 9.070011190933386|
|[12.0,42.0,14.8,7...|  6.61735744688879|
|[12.0,91.0,20.32,...| 9.328761223721694|
|[12.0,138.0,31.14...|13.094707673934701|
|[14.0,33.0,4.9,5....|3.1432896382182385|
|[14.0,63.0,14.4,7...| 6.776150948504927|
|[14.0,77.104,20.0...| 8.802733802823994|
|[14.0,83.0,17.5,9...| 9.234807503748625|
|[14.0,138.0,31.14...|13.067227749871881|
+--------------------+------------